based on the naive bayes implemented in class for sentiment analysis

dataset : train + test (80:20)

Task: binary
Labels: 0 (not ironic), 1 (ironic)
File format: a tab-separated file with one line per tweet containing per line the tweet index, a binary classification label, and the tweet text.

1) cleaned version 
without hashtags & explained emoji

## Setting the environement & Preparing the dataset

In [21]:
import numpy as np
import random

from sklearn import datasets
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

import pandas as pd

In [22]:
DATA_DIR = 'D:\\documents\\_KAIST\\s4\\NLP\\projet\\data\\SemEval2018-Task3-master\\datasets\\'
DATA_TRAIN = DATA_DIR + 'train\\SemEval2018-T3-train-taskA.txt'
DATA_TEST = DATA_DIR + 'test_TaskA\\SemEval2018-T3_input_test_taskA.txt'
LABEL_TEST = DATA_DIR + 'goldtest_TaskA\\SemEval2018-T3_gold_test_taskA_emoji.txt'
#print(DATA_TRAIN)
# headers are given by the first row. conversion into a csv file
train = pd.read_table(DATA_TRAIN, header=0)
"train data converted into a panda DataFrame"
test = pd.read_table(DATA_TEST, header=0)
testlabel = pd.read_table(LABEL_TEST, header=0)
"test data converted into a panda DataFrame"

'test data converted into a panda DataFrame'

## Training a Naive Bayes classifier

In [23]:
#instantiate a TfidfVectorizer object
#with word features, no stop words, lowercased documents, some ignored too frequent terms (>0.9), no maximum features,  
#with smooth idf weight, sublinear tf scaling and enabled inverse_document_frequency reweighting, 
#normalized with l1 norm
tfidfvectorizer = TfidfVectorizer()
    
#instantiate a MultinomialNB object
#that learn the class prior probabilities from the train dataset
#with an additive Laplace smotthing parameter alpha = 1.0

mnb = MultinomialNB()

In [24]:
#How to handle dataframes
#print(train.columns)
#print(train['Tweet text'])

#print(train['Label'])
print(test.columns)

Index(['tweet index', 'tweet text'], dtype='object')


In [25]:
def train_nb(train_data, train_target):
    #parameters: 
    ##the train dataset
    ##the classification labels of the train set
    #returns: an object of the class MultinomialNB, containing the probabilities needed to classify new instances
    ###
   
    #tokenize and build vocabulary on the train set
    #encode the raw documents into a sparse matrix X[n_sample, n_features]
    X_train_tfidf = tfidfvectorizer.fit_transform(train_data)
    
    #fit a Multinomial Naive Bayes classifier on the train set of targetet features
    mnb_classif = mnb.fit(X_train_tfidf, train_target)
    print("clf trained")
    return mnb_classif

## Classifying a new document


In [26]:
#taking a randomly chosen sample of the test set
row = random.choice(test['tweet index'])
#print(row)[
rd_data = test.loc[row,:]
rd_data = [rd_data['tweet text']]
print(rd_data)
print(type(rd_data)) 

['Day ninth December to have a pint at the boars head| really still in bed :smiling_face_with_smiling_eyes: http://t.co/H4XoH4jpwc']
<class 'list'>


In [27]:
def classify_nb(classifier, document):
    #parameters:
    ##classifier: an object of the class MultinomialNB, containing the probabilities needed to classify a new instance
    ##document: the text file that we want to classify, already tokenized 
    #returns: the prediction of the classifier
    
    #tokenize the test sample
    #encode the raw documents into a sparse matrix 
    X_test_sample_tfidf = tfidfvectorizer.transform(document)
    #print(X_test_tfidf)

    #predictions on the test set
    predicted = classifier.predict(X_test_sample_tfidf)
    print("prediction :")
    
    return predicted

In [28]:
trained = train_nb(train['Tweet text'], train['Label'])

classify_nb(trained, rd_data)

clf trained
prediction :


array([0], dtype=int64)

## Evaluating the classifier

In [31]:
%%time
#tokenize the test set
#encode the raw documents into a sparse matrix 
X_test_tfidf = tfidfvectorizer.transform(test['tweet text'])
#print(X_test_tfidf)

#predictions on the test set
predicted = trained.predict(X_test_tfidf)

#performance of the NB classifier
print(classification_report(testlabel['Label'], predicted, target_names = ['ironic','non ironic']))

acc = accuracy_score(testlabel['Label'], predicted)
print(acc)

             precision    recall  f1-score   support

     ironic       0.77      0.60      0.67       473
 non ironic       0.54      0.73      0.62       311

avg / total       0.68      0.65      0.65       784

0.6505102040816326
Wall time: 20.1 ms
